In [1]:
# ── 1. Setup ─────────────────────────────────────────────────────
import pystac_client
import planetary_computer
import xarray as xr
import pandas as pd
import numpy as np
from dask.distributed import Client, LocalCluster

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
# Optional: Monitor Dask
cluster = LocalCluster()
client = Client(cluster)

Opens a dashboard at http://localhost:8787

In [3]:
# ── 2. Load STAC GridMET catalog ────────────────────────────────
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
asset = catalog.get_collection("gridmet").assets["zarr-abfs"]


In [4]:
## ── 3. Open dataset lazily ───────────────────────────────────────
#ds = xr.open_zarr(
#    asset.href,
#    chunks={},  # Dask auto-chunking
#    storage_options=asset.extra_fields["xarray:storage_options"],
#    **asset.extra_fields["xarray:open_kwargs"]
#)

In [5]:
ds = xr.open_zarr(
    asset.href,
    chunks={"time": 1, "lat": 50, "lon": 50},  # force small local chunks
    storage_options=asset.extra_fields["xarray:storage_options"],
    **asset.extra_fields["xarray:open_kwargs"]
)

c:\Users\lucas\anaconda3\Lib\site-packages\xarray\core\dataset.py:265: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
c:\Users\lucas\anaconda3\Lib\site-packages\xarray\core\dataset.py:265: UserWarning: The specified chunks separate the stored chunks along dimension "lat" starting at index 50. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
c:\Users\lucas\anaconda3\Lib\site-packages\xarray\core\dataset.py:265: UserWarning: The specified chunks separate the stored chunks along dimension "lon" starting at index 50. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


In [6]:
# ── 4. Filter region and year (small for local!) ─────────────────
vars_to_use = [
    "air_temperature",  # target
    "relative_humidity",
    "wind_speed",
    "precipitation_amount"
]

ds_2020 = ds[vars_to_use].sel(
    time=slice("2020-01-01", "2020-01-01"),
    #lat=slice(49, 50),  # small box
    #lon=slice(-96, -95)
)

In [7]:
ds_2020

<xarray.Dataset>
Dimensions:               (time: 1, lat: 585, lon: 1386)
Coordinates:
  * lat                   (lat) float64 49.4 49.36 49.32 ... 25.15 25.11 25.07
  * lon                   (lon) float64 -124.8 -124.7 -124.7 ... -67.1 -67.06
  * time                  (time) datetime64[ns] 2020-01-01
Data variables:
    air_temperature       (time, lat, lon) float32 dask.array<chunksize=(1, 50, 50), meta=np.ndarray>
    relative_humidity     (time, lat, lon) float32 dask.array<chunksize=(1, 50, 50), meta=np.ndarray>
    wind_speed            (time, lat, lon) float32 dask.array<chunksize=(1, 50, 50), meta=np.ndarray>
    precipitation_amount  (time, lat, lon) float32 dask.array<chunksize=(1, 50, 50), meta=np.ndarray>
Attributes: (12/19)
    Conventions:                CF-1.6
    author:                     John Abatzoglou - University of Idaho, jabatz...
    coordinate_system:          EPSG:4326
    date:                       02 July 2019
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_bounds_crs:      EPSG:4326
    ...                         ...
    geospatial_lon_units:       decimal_degrees east
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...

In [8]:
# First, check that all variables have same dims
for var in vars_to_use:
    assert ds_2020[var].dims == ("time", "lat", "lon")

Obtain non-null values

In [9]:
# Now construct a valid mask over shared shape
combined_mask = xr.ones_like(ds_2020[vars_to_use[0]], dtype=bool)
for var in vars_to_use:
    combined_mask = combined_mask & ds_2020[var].notnull()

In [10]:
# Now apply the combined mask per variable
masked_data = {var: ds_2020[var].where(combined_mask) for var in vars_to_use}
ds_masked = xr.Dataset(masked_data)

In [11]:
ds_masked

<xarray.Dataset>
Dimensions:               (lat: 585, lon: 1386, time: 1)
Coordinates:
  * lat                   (lat) float64 49.4 49.36 49.32 ... 25.15 25.11 25.07
  * lon                   (lon) float64 -124.8 -124.7 -124.7 ... -67.1 -67.06
  * time                  (time) datetime64[ns] 2020-01-01
Data variables:
    air_temperature       (time, lat, lon) float32 dask.array<chunksize=(1, 50, 50), meta=np.ndarray>
    relative_humidity     (time, lat, lon) float32 dask.array<chunksize=(1, 50, 50), meta=np.ndarray>
    wind_speed            (time, lat, lon) float32 dask.array<chunksize=(1, 50, 50), meta=np.ndarray>
    precipitation_amount  (time, lat, lon) float32 dask.array<chunksize=(1, 50, 50), meta=np.ndarray>

In [12]:
# Convert to DataFrame
df = ds_masked.to_dataframe()

2025-06-03 12:34:20,203 - distributed.protocol.core - CRITICAL - Failed to Serialize
Traceback (most recent call last):
  File "c:\Users\lucas\anaconda3\Lib\site-packages\distributed\protocol\core.py", line 109, in dumps
    frames[0] = msgpack.dumps(msg, default=_encode_default, use_bin_type=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lucas\anaconda3\Lib\site-packages\msgpack\__init__.py", line 35, in packb
    return Packer(**kwargs).pack(o)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lucas\anaconda3\Lib\site-packages\msgpack\fallback.py", line 885, in pack
    self._pack(obj)
  File "c:\Users\lucas\anaconda3\Lib\site-packages\msgpack\fallback.py", line 861, in _pack
    self._pack(obj[i], nest_limit - 1)
  File "c:\Users\lucas\anaconda3\Lib\site-packages\msgpack\fallback.py", line 864, in _pack
    return self._pack_map_pairs(
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lucas\anaconda3\Lib\site-packages\msgpack

CancelledError: ('transpose-4f69a0067977a4c7f7775096168299d2', 1, 5, 0)

Modelling and FE

In [13]:
# ── 7. Feature Engineering ───────────────────────────────────────
X = df[["relative_humidity", "wind_speed", "precipitation_amount"]]
y = df["air_temperature"]

In [14]:
# ── 8. Train/Test Split ──────────────────────────────────────────
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# ── 9. Train ML Model ────────────────────────────────────────────
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

ValueError: Input y contains NaN.

In [ ]:
# ── 10. Evaluate ─────────────────────────────────────────────────
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {rmse:.3f} °C")